# Infretis Report [Template]

Infretis simulations generate a lot of data. Here we provide a standard template illustrating typical analysis that can be done.

#### Set the MD timestep and time unit utilized for the infretis simulation

`infretis` does not explicitly consider the md timestep or unit, and rather just deals with subcycles..

In [ ]:
### Usually for the following engines:
# Gromacs: md_dt = 0.002, units = "ps"
# CP2K:    md_dt = 0.5,   units = "fs"

md_dt = 1         
units = "internal"

#### Load in the typical infretis output

Data automtaically generated:
* `infretis.toml` : The toml file containing simulation settings. Needed for running a infretis simulation.
* `infretis_data.txt` : The swapping and high acceptance weights for the generated paths throughout an infretis simulation.
* `sim.log` : The log file for an infretis simulation.
* `load/` : The folder where accepted trajectories and their associated order.txt (etc) are saved to.

Standard wham analysis through running the `inftools` command: ```inft wham -h```
* wham : The wham folder containing the kinetics and (and free energies)

NB: The default ```inft wham -lamres``` value is kind of bad, so you try to chose a smaller value


In [ ]:
# standard imports
import os
import numpy as np
import matplotlib.pyplot as plt

from inftools.misc.infinit_helper import read_toml
from inftools.tistools.calc_simtime import calc_simtime
from inftools.misc.data_helper import data_reader

print("\nreport lies in ", os.getcwd())

# path to infretis data files/folders, to be changed if needed
fpaths = {
    "toml": "../infretis.toml",
    "data": "../infretis_data.txt",
    "log": "../sim.log",
    "load": "../load/",
    "wham": "../wham/",
}

# check which exists
valid = {key: os.path.exists(value) for key, value in fpaths.items()}

print("\nSimulation data:")
# print standard information
if valid["toml"]:
    toml = read_toml(fpaths["toml"])
    intfs = toml["simulation"]["interfaces"]
    print("Number of ensembles:", len(intfs), f", from 000 to {len(intfs)-1:03.0f}")
    subcycles = toml["engine"]["subcycles"]
    unitconv = subcycles*md_dt

if valid["data"]:
    data = data_reader(fpaths["data"])
    # count reactive
    rcnt = 0
    for path in data:
        if float(path["max_op"]) > intfs[-1]:
            rcnt += 1
    print(f"Sampled paths: {len(data)}, reactive: {rcnt}")

if valid["log"]:
    simtime, restarts = calc_simtime(fpaths["log"])
    print(f"Simulation wall time: {simtime:.04f} days, restarts: {restarts}\n")


### Flux, Conditional Crossing Probability and Rate

Running estimates for the flux, pcross and rate can be generated from ```inft wham -h```

In [ ]:
# Flux, Pcross, Rate
if valid["wham"]:
    runav_flux = np.loadtxt("../wham/runav_flux.txt")
    runav_pcro = np.loadtxt("../wham/runav_Pcross.txt")
    runav_rate = np.loadtxt("../wham/runav_rate.txt")
    pcro = np.loadtxt("../wham/Pcross.txt")
    err_flux = np.loadtxt("../wham/errFLUX.txt")
    err_pcro = np.loadtxt("../wham/errPtot.txt")
    err_rate = np.loadtxt("../wham/errRATE.txt")

    fig, axs = plt.subplots(1, 3, figsize=(16, 3))

    # Figure 1
    for intf in intfs:
        axs[0].axvline(intf, color="k", alpha=0.2, zorder=1)
    axs[0].plot(pcro[:, 0], pcro[:, -1], zorder=2)
    axs[0].set_yscale("log")
    axs[0].set_title(r"Conditional Prossing Probability")
    axs[0].set_xlabel("Order Parameter")

    # Figure 2
    runav_flux[:, -1] /= unitconv
    runav_rate[:, -1] /= unitconv
    axs[1].axhline(runav_rate[-1, -1], alpha=0.2, color="C0")
    axs[1].plot(runav_rate[:, 0], runav_rate[:, -1], label=f"Rate [1/{units}]", color="C0")
    axs[1].axhline(runav_pcro[-1, -1], alpha=0.2, color="C1")
    axs[1].plot(runav_pcro[:, 0], runav_pcro[:, -1], label="Pcross", color="C1")
    axs[1].axhline(runav_flux[-1, -1], alpha=0.2, color="C2")
    axs[1].plot(runav_flux[:, 0], runav_flux[:, -1], label=f"Flux [1/{units}]", color="C2")
    axs[1].set_yscale("log")
    axs[1].set_title(r"Running Average estimates")
    axs[1].set_xlabel("Accepted Paths")
    axs[1].legend(bbox_to_anchor=(-0.15, 1.02, 1, 0.2), loc="lower left", mode="expand",
                  borderaxespad=0, fontsize=8, frameon=False)

    # Figure 3
    axs[2].axhline(err_rate[-1, -1], alpha=0.2, color="C0")
    axs[2].plot(err_rate[:, 0], err_rate[:, -1], label="Rate", color="C0")
    axs[2].axhline(err_pcro[-1, -1], alpha=0.2, color="C1")
    axs[2].plot(err_pcro[:, 0], err_pcro[:, -1], label="Pcross", color="C1")
    axs[2].axhline(err_flux[-1, -1], alpha=0.2, color="C2")
    axs[2].plot(err_flux[:, 0], err_flux[:, -1], label="Flux", color="C2")
    axs[2].set_yscale("log")
    axs[2].set_title(r"Running Block Error")
    axs[2].set_xlabel("Accepted Paths")
    axs[2].legend(bbox_to_anchor=(-0.15, 1.02, 1, 0.2), loc="lower left", mode="expand",
                  borderaxespad=0, fontsize=8, frameon=False)
    plt.show()

print(f"With a pcross {runav_pcro[-1, -1]:.04e} and flux {runav_flux[-1, -1]:.04e} [1/{units}], time step of {md_dt} {units}",)
print(f"rate is estimated to be {runav_rate[-1, -1]:.4e} [1/{units}]")
# to be... is the error

#### Individual Path Convergence

For [i^+] ensembles only.

In [ ]:
ploc_pcros = np.loadtxt("../wham/ploc_WHAM.txt")
ploc_runav = np.loadtxt("../wham/runav_ploc.txt")
ploc_err   = np.loadtxt("../wham/errploc.txt")

# for i, ens  in range(1, len(intfs)):
for i, ens in enumerate([f"[{i}^+]" for i in range(len(intfs)-1)], start=1):
    fig, axs = plt.subplots(1, 3, figsize=(16, 3))
    axs[0].plot(ploc_pcros[:, 0], ploc_pcros[:, i]/np.max(ploc_pcros[:, i]), zorder=5)
    axs[0].set_ylim([0, 1])
    axs[0].axvline(intfs[0], color="k")
    axs[0].axvline(intfs[-1], color="k")
    axs[0].axvline(intfs[i], color="k", alpha=0.4, ls="--")
    axs[0].set_xlabel(r"Order parameter ($\lambda$)")
    axs[0].set_title(f"{ens} Crossing probability")

    axs[1].plot(ploc_runav[:, 0], ploc_runav[:, i], zorder=5)
    axs[1].axhline(ploc_runav[-1, i], color="k", alpha=0.4, ls="--")
    axs[1].set_xlabel(r"Uniquely sampled path")
    axs[1].set_title(f"{ens} Running estimate")
    
    axs[2].plot(ploc_err[:, 0], ploc_err[:, i], zorder=5)
    half = int(len(ploc_err[:, i])/2)
    axs[2].axhline(np.average(ploc_err[half:, i]), color="k", alpha=0.4, ls="--")
    axs[2].set_xlabel(r"Block length")
    axs[2].set_title(f"{ens} Estimated error")
    plt.show()


#### Average ensemble path lenghts

In [ ]:
plens = np.loadtxt("../wham/pathlengths.txt")
plt.scatter(plens[:, 0], plens[:, 1]*subcycles*md_dt)
# plt.xticks(plens[:, 0], [f"{i:03.0f}" for i in range(len(intfs))])
plt.xticks(plens[:, 0], ["[0^-]"] + [f"[{i}^+]" for i in range(len(intfs)-1)])
plt.xlabel("Ensembles")
plt.ylabel(f"Path length [{units}]")

### Replica Exchange and Flow

Better decorrelation and path space exploration is enabled through the "flow" of replica across path ensembles throughout an infretis simulation.



In [ ]:
# Investigate flow of replica through ensembles:
from inftools.tistools.flow_op import calc_flow_op

replica = 0 # any value between 0 to -1
for rep in range(len(intfs)):
    print("replica", rep)
    _ = calc_flow_op(rep=rep, toml=fpaths["toml"], log=fpaths["log"], load=fpaths["load"])

### Source

[1] heh

[2]

[3]